## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [PHI leakage detection for DICOM ](https://aws.amazon.com/marketplace/pp/prodview-h32wcyfp7dlac)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## PHI leakage detection for DICOM 


- **Model**: `dicom_deid_pixels_platform_en`
- **Model Description**: This pipeline flags PHI entities in both DICOM images and metadata to support compliance monitoring and violation reporting.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "dicom-deid-pixels-platform-en"

real_time_inference_instance_type = "ml.m4.2xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

In [10]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [11]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

------------!

Once endpoint has been created, you would be able to perform real-time inference.

In [12]:
def invoke_realtime_endpoint(file):
    with open(file, "rb") as f:
        dicome_data = f.read()

    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType='application/octet-stream',
        Accept='application/json',
        Body=dicome_data
    )

    response_body = response["Body"].read().decode("utf-8")
    result = json.loads(response_body)

    return result

### Initial setup

In [ ]:
!wget --no-check-certificate https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/products/sagemaker/models/dicom_deid_pixels_platform_en/inputs/batch/example_input_1.dcm
!wget --no-check-certificate https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/products/sagemaker/models/dicom_deid_pixels_platform_en/inputs/batch/example_input_2.dcm

### JSON

#### Example 1

In [13]:
response = invoke_realtime_endpoint("example_input_1.dcm")

In [14]:
response

{'raw_document': [{'text': '‘Name:\n : Mccoy\n Medical\n Clinic\n Institution\n ©: 2023 jul. 02\n Study\n oe\n Patient\n Name: iura\n  Scrobola\n fe\n 30\n “10/08/1992\n',
   'begin': 0,
   'end': 126}],
 'document': [{'text': '‘Name: : Mccoy Medical Clinic Institution ©: 2023 jul. 02 Study oe Patient Name: iura Scrobola fe 30 “10/08/1992',
   'begin': 0,
   'end': 111}],
 'sentence': [{'text': '‘Name: : Mccoy Medical Clinic Institution ©: 2023 jul. 02 Study oe Patient Name: iura Scrobola fe 30 “10/08/1992',
   'begin': 0,
   'end': 111}],
 'ner_predictions': [{'ner_chunk': ': Mccoy',
   'begin': 7,
   'end': 14,
   'ner_label': 'UNK'},
  {'ner_chunk': 'Medical', 'begin': 15, 'end': 22, 'ner_label': 'UNK'},
  {'ner_chunk': 'Clinic', 'begin': 23, 'end': 29, 'ner_label': 'UNK'},
  {'ner_chunk': '2023', 'begin': 45, 'end': 48, 'ner_label': 'DATE'},
  {'ner_chunk': 'Study', 'begin': 58, 'end': 63, 'ner_label': 'UNK'},
  {'ner_chunk': 'Patient', 'begin': 67, 'end': 74, 'ner_label': 'UNK'},


#### Example 2

In [15]:
response = invoke_realtime_endpoint("example_input_2.dcm")

In [16]:
response

{'raw_document': [{'text': 'Institution\n  Ball.\n ‘Name:\n and\n  Community\n Clinic\n  “Bate:\n 2023\n Mar27\n “Patient\n “1p: 571403367\n  TANNA.\n ‘Name:\n  ‘DOB:\n 03/11/1948\n',
   'begin': 0,
   'end': 137}],
 'document': [{'text': 'Institution Ball. ‘Name: and Community Clinic “Bate: 2023 Mar27 “Patient “1p: 571403367 TANNA. ‘Name: ‘DOB: 03/11/1948',
   'begin': 0,
   'end': 117}],
 'sentence': [{'text': 'Institution Ball.', 'begin': 0, 'end': 16},
  {'text': '‘Name: and Community Clinic “Bate: 2023 Mar27 “Patient “1p: 571403367 TANNA.',
   'begin': 18,
   'end': 93},
  {'text': '‘Name: ‘DOB: 03/11/1948', 'begin': 95, 'end': 117}],
 'ner_predictions': [{'ner_chunk': 'Ball.',
   'begin': 13,
   'end': 18,
   'ner_label': 'UNK'},
  {'ner_chunk': 'and', 'begin': 26, 'end': 29, 'ner_label': 'UNK'},
  {'ner_chunk': 'Community', 'begin': 31, 'end': 40, 'ner_label': 'UNK'},
  {'ner_chunk': 'Clinic', 'begin': 41, 'end': 47, 'ner_label': 'UNK'},
  {'ner_chunk': '2023', 'begin': 56, 'end

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [17]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [18]:
validation_file_name1 = "example_input_1.dcm"
validation_file_name2 = "example_input_2.dcm"

validation_input_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/"
validation_output_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/"

def upload_dicom_to_s3(local_file_path):
    base_file_name = os.path.basename(local_file_path)
    validation_input_file_path = f"{model_name}/validation-input/batch/{base_file_name}"
    s3_client.upload_file(local_file_path, s3_bucket, validation_input_file_path)

In [19]:
upload_dicom_to_s3(validation_file_name1)
upload_dicom_to_s3(validation_file_name2)

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_path
)

transformer.transform(validation_input_path, content_type='application/octet-stream')
transformer.wait()

In [21]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [22]:
retrieve_json_output_from_s3(validation_file_name1)

{'raw_document': [{'text': '‘Name:\n : Mccoy\n Medical\n Clinic\n Institution\n ©: 2023 jul. 02\n Study\n oe\n Patient\n Name: iura\n  Scrobola\n fe\n 30\n “10/08/1992\n',
   'begin': 0,
   'end': 126}],
 'document': [{'text': '‘Name: : Mccoy Medical Clinic Institution ©: 2023 jul. 02 Study oe Patient Name: iura Scrobola fe 30 “10/08/1992',
   'begin': 0,
   'end': 111}],
 'sentence': [{'text': '‘Name: : Mccoy Medical Clinic Institution ©: 2023 jul. 02 Study oe Patient Name: iura Scrobola fe 30 “10/08/1992',
   'begin': 0,
   'end': 111}],
 'ner_predictions': [{'ner_chunk': ': Mccoy',
   'begin': 7,
   'end': 14,
   'ner_label': 'UNK'},
  {'ner_chunk': 'Medical', 'begin': 15, 'end': 22, 'ner_label': 'UNK'},
  {'ner_chunk': 'Clinic', 'begin': 23, 'end': 29, 'ner_label': 'UNK'},
  {'ner_chunk': '2023', 'begin': 45, 'end': 48, 'ner_label': 'DATE'},
  {'ner_chunk': 'Study', 'begin': 58, 'end': 63, 'ner_label': 'UNK'},
  {'ner_chunk': 'Patient', 'begin': 67, 'end': 74, 'ner_label': 'UNK'},


In [23]:
retrieve_json_output_from_s3(validation_file_name2)

{'raw_document': [{'text': 'Institution\n  Ball.\n ‘Name:\n and\n  Community\n Clinic\n  “Bate:\n 2023\n Mar27\n “Patient\n “1p: 571403367\n  TANNA.\n ‘Name:\n  ‘DOB:\n 03/11/1948\n',
   'begin': 0,
   'end': 137}],
 'document': [{'text': 'Institution Ball. ‘Name: and Community Clinic “Bate: 2023 Mar27 “Patient “1p: 571403367 TANNA. ‘Name: ‘DOB: 03/11/1948',
   'begin': 0,
   'end': 117}],
 'sentence': [{'text': 'Institution Ball.', 'begin': 0, 'end': 16},
  {'text': '‘Name: and Community Clinic “Bate: 2023 Mar27 “Patient “1p: 571403367 TANNA.',
   'begin': 18,
   'end': 93},
  {'text': '‘Name: ‘DOB: 03/11/1948', 'begin': 95, 'end': 117}],
 'ner_predictions': [{'ner_chunk': 'Ball.',
   'begin': 13,
   'end': 18,
   'ner_label': 'UNK'},
  {'ner_chunk': 'and', 'begin': 26, 'end': 29, 'ner_label': 'UNK'},
  {'ner_chunk': 'Community', 'begin': 31, 'end': 40, 'ner_label': 'UNK'},
  {'ner_chunk': 'Clinic', 'begin': 41, 'end': 47, 'ner_label': 'UNK'},
  {'ner_chunk': '2023', 'begin': 56, 'end

In [24]:
model.delete_model()

INFO:sagemaker:Deleting model with name: dicom-deid-pixels-platform-en-2025-06-27-07-23-00-712


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

